<a href="https://colab.research.google.com/github/YogeshParihar1311/Deep_Learning/blob/main/RESUME_CUSTOME_NER_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.6 MB/s eta 0:00:00


In [4]:
import os
import PyPDF2

source_folder = 'Custom_NER/resume_dataset/pdf_format'
combined_txt_path = 'Custom_NER/resume_dataset/pdf_format/combined_resumes.txt'

# Create a list to store text from all PDF files
all_pdf_text = []

# Loop through PDF files in the source folder
for filename in os.listdir(source_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(source_folder, filename)

        # Extract text from PDF
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            pdf_text = ' '.join(page.extract_text() for page in pdf_reader.pages)
            all_pdf_text.append(pdf_text)

# Combine all PDF text into a single string
combined_text = '\n\n'.join(all_pdf_text)  # Adding double newline separator

# Save combined text to a single TXT file
with open(combined_txt_path, 'w', encoding='utf-8') as combined_txt_file:
    combined_txt_file.write(combined_text)

In [20]:
pip install spacy_transformers

In [18]:
# install any necessary packages
!pip install -U spacy
!pip install spacy_transformers

# Install below packages if error occurs while installing above packages
#!pip install -U pip setuptools
# !pip install typing-extensions==4.6.1

In [5]:
# Import the necessary libraries and mount Google Drive to access project files
# from google.colab import drive
# drive.mount('/content/drive')

# # Change the working directory to the project folder
# %cd "/content/drive/MyDrive/Custom_NER"

# Import required libraries and install any necessary packages
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

# Check the installed version of spaCy
spacy.__version__

# Check GPU information
!nvidia-smi

Fri Apr  5 13:55:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
# Load the annotated data from a JSON file
cv_data = json.load(open('Custom_NER/annotations_dataset/annotations.json','r'))

# Display the number of items in the dataset
print(len(cv_data))

# Display the first item in the dataset
cv_data[0]

50


['Your Name\r', {'entities': []}]

In [12]:
# Initialize spaCy configuration files by copying from base_config to config.cfg
# !python -m spacy init fill-config /"your_file_path"/base_config.cfg /"your_file_path"/config/config.cfg
!python -m spacy init fill-config Custom_NER/config/base_config.cfg Custom_NER/config/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
Custom_NER/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [14]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.2)

# Display the number of items in the training and testing sets
len(train), len(test)

# Open a file to log errors during annotation processing
file = open('Custom_NER/trained_models/train_file.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('Custom_NER/trained_models/train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('Custom_NER/trained_models/test_data.spacy')

# Close the error log file
file.close()

100%|██████████| 10/10 [00:00<00:00, 1652.41it/s]


In [15]:
# Train a spaCy NER model using the provided configuration and data
!python -m spacy train Custom_NER/config/config.cfg  --output Custom_NER/trained_models/output  --paths.train Custom_NER/trained_models/train_data.spacy  --paths.dev Custom_NER/trained_models/test_data.spacy --gpu-id 0

✔ Created output directory: Custom_NER/trained_models/output
ℹ Saving to output directory: Custom_NER/trained_models/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 120kB/s]
config.json: 100% 481/481 [00:00<00:00, 3.13MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 9.85MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 71.0MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 17.1MB/s]
model.safetensors: 100

In [22]:
# install any necessary packages
!pip install fitz
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 38.8 MB/s eta 0:00:00


In [5]:
# Import the spaCy library
import spacy

# Load the trained spaCy NER model from the specified path
nlp = spacy.load('Custom_NER/trained_models/output/model-best')

# Import necessary libraries for PDF processing
import sys
import fitz

# Specify the path to the PDF file containing the resume
fname = 'Custom_NER/test_dataset/resume_juanjosecarin.pdf'

# Open the PDF document using PyMuPDF (fitz)
doc = fitz.open(fname)

# Initialize an empty string to store the extracted text from the PDF
text = " "

# Iterate through each page in the PDF and concatenate the text
for page in doc:
  text = text + str(page.get_text())

# Display the extracted text
print(text)

 1 of 2 
Juan Jose Carin 
Data Scientist 
 
Mountain View, CA 94041 
 
650-336-4590 | juanjose.carin@gmail.com 
 
linkedin.com/in/juanjosecarin | 
juanjocarin.github.io 
 
Professional Profile 
Passionate about data analysis and experiments, mainly focused on user behavior, experience, and engagement, with a solid 
background in data science and statistics, and extensive experience using data insights to drive business growth. 
Education
2016 
University of California, Berkeley 
Master of Information and Data Science 
GPA: 3.93
 
 
 
Relevant courses: 
• Machine Learning 
• Machine Learning at Scale 
• Storing and Retrieving Data 
• Field Experiments 
• Applied Regression and Time Series 
Analysis 
• Exploring and Analyzing Data 
• Data Visualization and 
Communication 
• Research Design and Applications for 
Data Analysis 
2014 
Universidad Politécnica de Madrid 
M.S. in Statistical and Computational Information Processing 
GPA: 3.69
 
 
 
Relevant courses:  
• Data Mining 
• Multivar

In [6]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text)

# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)

linkedin.com/in/juanjosecarin   ->>>>   NAME
data analysis   ->>>>   SKILL
data science   ->>>>   SKILL
University of California,   ->>>>   DEGREE
Master of Information and   ->>>>   DEGREE
Machine Learning   ->>>>   SKILL
Machine Learning   ->>>>   SKILL
Applied Regression   ->>>>   SKILL
Data Visualization   ->>>>   SKILL
Data Mining   ->>>>   SKILL
Multivariate Analysis   ->>>>   SKILL
Time Series   ->>>>   SKILL
Neural Networks   ->>>>   SKILL
Regression and Prediction   ->>>>   SKILL
Optimization Techniques   ->>>>   SKILL
Monte Carlo   ->>>>   SKILL
Numerical Methods   ->>>>   SKILL
Stochastic Models   ->>>>   SKILL
Bayesian Networks   ->>>>   SKILL
Radio communication   ->>>>   SKILL
radar   ->>>>   SKILL
mobile   ->>>>   SKILL
Statistics   ->>>>   SKILL
Big Data   ->>>>   SKILL
Visualization   ->>>>   SKILL
R   ->>>>   SKILL
Python   ->>>>   SKILL
SQL   ->>>>   SKILL
Hadoop   ->>>>   SKILL
Hive   ->>>>   SKILL
MrJob 
Tableau   ->>>>   SKILL
Git   ->>>>   SKILL
AWS   ->>>>   SKI